# 6.2 Стекинг

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [3]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2


In [4]:
df.shape

(10000, 55)

In [5]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [6]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [7]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [8]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test


In [9]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test


In [10]:
from sklearn.metrics import f1_score
cv = KFold(n_splits=10, shuffle=True, random_state=42)

#Используемая метрика
def compute_metric(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)



In [11]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.06s/it]


In [12]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)


0.795443

## Задание 6.6.1

В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.

Логика решения:

1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);

2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);

3) После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.

За основу нужно взять следующий код:

In [13]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test  = np.zeros((len(X_test), n_classes), dtype=np.float32)
    n_folds = 0.0
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test = X_meta_test + folded_clf.predict_proba(X_test)
        n_folds = n_folds +1.0

    X_meta_test = X_meta_test / n_folds

    return X_meta_train, X_meta_test

## Задание 6.6.2

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000

логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000

случайный лес из 300 деревьев

градиентный бустинг из 200 деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [14]:
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr', random_state=42),
    LogisticRegression(C=0.001, penalty='l2', solver='saga', max_iter=2000, multi_class='multinomial', random_state=42),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42),
    GradientBoostingClassifier(n_estimators=200, random_state=42)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.18s/it]


In [15]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto', random_state=42)
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.790437

## Задание 6.6.3

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

случайный лес из 300 деревьев

случайный лес из 200 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [16]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=200, n_jobs=-1)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.34s/it]


In [17]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto')
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.79471

## Задание 6.6.4

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

метод ближайшего соседа (k-NN) со стандартными параметрами

случайный лес из 300 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [18]:
from sklearn.neighbors import KNeighborsClassifier
stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.43s/it]


In [19]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto')
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.808068

## Задание 6.6.5

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000

метод ближайшего соседа со стандартными параметрами

случайный лес из 300 экстремальных деревьев

AdaBoost со стандартными параметрами

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [20]:
from sklearn.ensemble import AdaBoostClassifier
stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='none', solver='saga', max_iter=2000, multi_class='ovr'),
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1),
    AdaBoostClassifier()  
], cover_X_train, cover_X_test, cover_y_train.values, cv)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnin

In [21]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto')
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.814389

## Задание 6.6.6

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

случайный лес из 300 деревьев

случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов). Для корректной работы необходимо подправить код в функции compute_meta_feature. 

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [22]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

from sklearn.ensemble import AdaBoostClassifier
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1),
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.28s/it]


In [23]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto')
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.797584

## Задание 6.6.7

В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [24]:
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
from sklearn.ensemble import AdaBoostClassifier
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1),
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:28<00:00, 14.11s/it]


In [25]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs',multi_class='auto')
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.796887

## Задание 6.6.8 

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [26]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.ensemble import AdaBoostClassifier
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1),
], cover_X_train, cover_X_test, cover_y_train.values, cv)
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  4.00s/it]


In [27]:
np.random.seed(42)
clf = RandomForestClassifier()
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.769651

## Задание 6.6.9

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [28]:
np.random.seed(42)
clf = KNeighborsClassifier()
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.782338

## Задание 6.6.10

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [29]:
np.random.seed(42)
clf = GradientBoostingClassifier()
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.791133

## Задание 6.6.11

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24

случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.

Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [30]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
from sklearn.ensemble import AdaBoostClassifier
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1, criterion = 'gini', max_depth = 24),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1),
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.60s/it]


In [31]:
np.random.seed(42)
clf = ExtraTreesClassifier(n_estimators=100, n_jobs=-1)
compute_metric(clf, stacked_features_train, cover_y_train, stacked_features_test,cover_y_test)

0.745432

## Задание 6.6.12

Обучите на тренировочной выборке следующие алгоритмы:

случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24

случайный лес из 300 экстремальных деревьев

логистическую регрессию со стандартными параметрами

Усредните их ответы на тестовой выборке и посчитайте качество аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [33]:
clf_rf = RandomForestClassifier(n_estimators=300, n_jobs=-1, criterion = 'gini', max_depth = 24, random_state=42)
clf_ef = ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
clf_logr = LogisticRegression(random_state=42)
clf_rf.fit(cover_X_train,cover_y_train)
clf_ef.fit(cover_X_train,cover_y_train)
clf_logr.fit(cover_X_train,cover_y_train)
y_predict = (clf_rf.predict(cover_X_test)+clf_ef.predict(cover_X_test)+clf_logr.predict(cover_X_test))/3.0
y_predict = y_predict.astype(np.int64)
np.round(f1_score(cover_y_test, y_predict, average='macro'), 6)

0.78484